<a href="https://colab.research.google.com/github/MekaRaptor/Graduation_Project/blob/main/FPN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet50
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import os

drive_path = "/content/drive/MyDrive/"  # ZIP dosyanın bulunduğu klasör
os.listdir(drive_path)  # Klasördeki dosyaları listele


['Classroom',
 'Alper_Ağırman_201180010 (1).txt',
 'Alper_Ağırman_201180010.txt',
 'Ceng307_201180010_HW1.sln',
 'Bilge Adam Data Science',
 'Untitled Diagram (4).drawio',
 'Untitled Diagram (3).drawio',
 'Document (1).sdr',
 'Untitled Diagram (2).drawio',
 'AI_Model - Made with Clipchamp_1718179352783.mp4',
 'Project Details',
 'Adsız sunu.gslides',
 'Untitled Diagram (1).drawio',
 'Neural_Network Documentation dokümanının çevrilmiş kopyası.gdoc',
 'Neural_Network Documentation.gdoc',
 'Literatür Taraması.gdoc',
 'Adsız doküman (2).gdoc',
 'Staj Raporu.gdoc',
 'Neural_Network Documentation (1).pdf',
 'staj2018-sablon (3).docx',
 'SRS_aciklamalar.gdoc',
 'SRS-Sablon.gdoc',
 'SDD_aciklamalar (1).gdoc',
 'SDD_aciklamalar.gdoc',
 'Untitled Diagram.drawio',
 'SPMP_Ornek.gdoc',
 'SDD-Sablon.gdoc',
 'week9.gdoc',
 'llm.gdoc',
 'clustering.gdoc',
 'cnn_rnn_gnn.gdoc',
 'NN.gdoc',
 'GAZI Midterm.gdoc',
 'Meet Recordings',
 'Bilgilendirme Metni (1).docx',
 'Bilgilendirme Metni.docx',
 'Do

In [5]:
import zipfile

zip_path = "/content/drive/MyDrive/Bitirme/Agriculture-Vision-2021.zip"  # ZIP dosyasının gerçek adını yaz
extract_path = "/content/veri_seti"  # Açılacağı yer

# ZIP dosyasını açma işlemi
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ ZIP dosyası başarıyla çıkarıldı.")


✅ ZIP dosyası başarıyla çıkarıldı.


In [6]:
dataset_path = "/content/veri_seti/Agriculture-Vision-2021"

# Ana klasörleri listeleyelim
folders = os.listdir(dataset_path)
print("📂 Veri Seti İçeriği:")
for folder in folders:
    print("-", folder)


📂 Veri Seti İçeriği:
- train
- val
- test


In [7]:
import os

# Ana klasör
dataset_path = "/content/veri_seti/Agriculture-Vision-2021"
train_path = os.path.join(dataset_path, "train")

# Alt klasör yolları
rgb_path = os.path.join(train_path, "images", "rgb")
nir_path = os.path.join(train_path, "images", "nir")
mask_path = os.path.join(train_path, "masks")  # ⬅️ Eksik olan burasıydı
label_path = os.path.join(train_path, "labels")

# Etiket sınıflarını klasör olarak alalım
label_classes = os.listdir(label_path)

print("📁 Etiket Sınıfları:")
for cls in label_classes:
    print("-", cls)


📁 Etiket Sınıfları:
- waterway
- endrow
- nutrient_deficiency
- drydown
- water
- storm_damage
- double_plant
- planter_skip
- weed_cluster


In [8]:
from PIL import Image

# Sınıf adları ve id'leri
CLASS_NAMES = [
    "double_plant",
    "drydown",
    "endrow",
    "nutrient_deficiency",
    "planter_skip",
    "storm_damage",
    "water",
    "waterway",
    "weed_cluster"
]
CLASS_NAME_TO_ID = {name: idx for idx, name in enumerate(CLASS_NAMES)}

def combine_label_masks(label_root, output_path, image_size=(512, 512)):
    os.makedirs(output_path, exist_ok=True)

    # Referans olarak bir sınıf klasöründen dosya adlarını al
    ref_class = CLASS_NAMES[0]
    file_list = sorted(os.listdir(os.path.join(label_root, ref_class)))

    for filename in file_list:
        combined_mask = np.zeros(image_size, dtype=np.uint8)

        for class_name in CLASS_NAMES:
            class_id = CLASS_NAME_TO_ID[class_name]
            mask_path = os.path.join(label_root, class_name, filename)

            if os.path.exists(mask_path):
                mask = np.array(Image.open(mask_path).convert("L"))
                binary_mask = mask > 127
                combined_mask[binary_mask] = class_id

        out_path = os.path.join(output_path, filename)
        Image.fromarray(combined_mask).save(out_path)

    print(f"✅ {output_path} klasörüne {len(file_list)} etiket maskesi oluşturuldu.")


In [9]:
# Train set için
combine_label_masks(
    label_root="/content/veri_seti/Agriculture-Vision-2021/train/labels",
    output_path="/content/veri_seti/Agriculture-Vision-2021/train/combined_labels"
)

# Validation set için
combine_label_masks(
    label_root="/content/veri_seti/Agriculture-Vision-2021/val/labels",
    output_path="/content/veri_seti/Agriculture-Vision-2021/val/combined_labels"
)


✅ /content/veri_seti/Agriculture-Vision-2021/train/combined_labels klasörüne 56944 etiket maskesi oluşturuldu.
✅ /content/veri_seti/Agriculture-Vision-2021/val/combined_labels klasörüne 18334 etiket maskesi oluşturuldu.


In [10]:
class AgricultureDataset(Dataset):
    def __init__(self, image_dir_rgb, image_dir_nir, label_dir, mask_dir, transform=None):
        self.image_dir_rgb = image_dir_rgb
        self.image_dir_nir = image_dir_nir
        self.label_dir = label_dir
        self.mask_dir = mask_dir
        self.transform = transform

        # Dosya isimlerinin köklerini al
        rgb_stems = {Path(f).stem for f in os.listdir(image_dir_rgb)}
        label_stems = {Path(f).stem for f in os.listdir(label_dir)}
        mask_stems = {Path(f).stem for f in os.listdir(mask_dir)}

        # Ortak stem'leri bul
        common_stems = sorted(list(rgb_stems & label_stems & mask_stems))

        # Bunları .jpg uzantılı olarak sakla (çünkü rgb dosyaları .jpg)
        self.filenames = [stem + ".jpg" for stem in common_stems]

        print(f"📊 Dataset'te kullanılabilir örnek sayısı: {len(self.filenames)}")

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        from pathlib import Path
        filename = self.filenames[idx]

        try:
            # Dosya yolları
            rgb_path = os.path.join(self.image_dir_rgb, filename)
            nir_path = os.path.join(self.image_dir_nir, filename)
            label_path = os.path.join(self.label_dir, Path(filename).stem + ".png")
            mask_path = os.path.join(self.mask_dir, Path(filename).stem + ".png")

            # RGB ve NIR oku
            rgb = cv2.imread(rgb_path)
            nir = cv2.imread(nir_path, cv2.IMREAD_GRAYSCALE)
            label = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)

            if rgb is None or nir is None or label is None:
                print(f"Skipping file {filename} due to unreadable input.")
                return None

            rgb = rgb[:, :, ::-1]  # BGR to RGB
            nir = np.expand_dims(nir, axis=-1)
            image = np.concatenate([rgb, nir], axis=-1)

            # Valid mask oku
            if os.path.exists(mask_path):
                mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                valid_mask = (mask_img > 127) if mask_img is not None else np.ones((512, 512), dtype=bool)
            else:
                valid_mask = np.ones((512, 512), dtype=bool)

            # Augmentations uygula
            if self.transform:
                augmented = self.transform(image=image, mask=label)
                image = augmented["image"]
                label = augmented["mask"]

            # DİKKAT: label artık Tensor!
            label = label.clone()  # Tensor'a müdahale için kopya al
            valid_mask_tensor = torch.from_numpy(valid_mask)
            label[~valid_mask_tensor] = 255  # ignore index

            return image, label.long()

        except Exception as e:
            print(f"Skipping file {filename} due to error: {e}")
            return None


In [11]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pathlib import Path


transform = A.Compose([
    A.Resize(512, 512),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Normalize(),
    ToTensorV2()
])

train_dataset = AgricultureDataset(
    image_dir_rgb="/content/veri_seti/Agriculture-Vision-2021/train/images/rgb",
    image_dir_nir="/content/veri_seti/Agriculture-Vision-2021/train/images/nir",
    label_dir="/content/veri_seti/Agriculture-Vision-2021/train/combined_labels",
    mask_dir="/content/veri_seti/Agriculture-Vision-2021/train/masks",
    transform=transform
)



📊 Dataset'te kullanılabilir örnek sayısı: 56944


In [12]:
import torch.nn as nn
import torchvision.models as models
import torch
import torch.nn.functional as F

class ConvBNReLU(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.01)
        )

    def forward(self, x):
        return self.conv(x)

class FPNDecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.deconv = nn.ConvTranspose2d(
            in_channels, out_channels, kernel_size=3, stride=2,
            padding=1, output_padding=1, bias=False
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.LeakyReLU(0.01)
        self.conv1x1 = nn.Conv2d(out_channels, out_channels, kernel_size=1, bias=False)

    def forward(self, x):
        x = self.deconv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.conv1x1(x)
        return x

class FPNModel(nn.Module):
    def __init__(self, num_classes=9, in_channels=4, backbone="resnet101"):
        super().__init__()
        assert backbone in ["resnet50", "resnet101"]

        # Load ResNet backbone
        resnet = getattr(models, backbone)(weights='IMAGENET1K_V1')
        if in_channels != 3:
            resnet.conv1 = nn.Conv2d(
                in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False
            )

        # Encoder layers
        self.layer1 = nn.Sequential(
            resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool, resnet.layer1
        )
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4

        # 🛠️ Layer4'ü dilated yap
        for name, module in self.layer4.named_modules():
            if isinstance(module, nn.Conv2d):
                if module.kernel_size == (3, 3):
                    module.dilation = (4, 4)
                    module.padding = (4, 4)
                    module.stride = (1, 1)
            if isinstance(module, nn.Conv2d) and module.kernel_size == (1, 1):
                module.stride = (1, 1)
            if isinstance(module, nn.Sequential):
                for subname, submodule in module.named_modules():
                    if isinstance(submodule, nn.Conv2d):
                        submodule.stride = (1, 1)

        # Lateral 1x1 convolutions
        self.lat1 = nn.Conv2d(256, 128, kernel_size=1)
        self.lat2 = nn.Conv2d(512, 128, kernel_size=1)
        self.lat3 = nn.Conv2d(1024, 128, kernel_size=1)
        self.lat4 = nn.Conv2d(2048, 128, kernel_size=1)

        # Decoder upsample blocks
        self.dec3 = FPNDecoderBlock(128, 128)
        self.dec2 = FPNDecoderBlock(128, 128)
        self.dec1 = FPNDecoderBlock(128, 128)

        # Final fusion
        self.final_conv = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.01),
            nn.Conv2d(128, num_classes, kernel_size=1)
        )

    def forward(self, x):
        c1 = self.layer1(x)
        c2 = self.layer2(c1)
        c3 = self.layer3(c2)
        c4 = self.layer4(c3)

        p4 = self.lat4(c4)

        p3 = self.dec3(p4)
        p3 = F.interpolate(p3, size=c3.shape[2:], mode='bilinear', align_corners=True)
        p3 = p3 + self.lat3(c3)

        p2 = self.dec2(p3)
        p2 = F.interpolate(p2, size=c2.shape[2:], mode='bilinear', align_corners=True)
        p2 = p2 + self.lat2(c2)

        p1 = self.dec1(p2)
        p1 = F.interpolate(p1, size=c1.shape[2:], mode='bilinear', align_corners=True)
        p1 = p1 + self.lat1(c1)

        out = self.final_conv(p1)
        out = F.interpolate(out, size=x.shape[2:], mode='bilinear', align_corners=True)  # 👈 giriş boyutuna dön
        return out




In [13]:
model = FPNModel(num_classes=9, in_channels=4, backbone="resnet101").to("cuda")
print("✅ FPN modeli başarıyla oluşturuldu.")


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 206MB/s]


✅ FPN modeli başarıyla oluşturuldu.


In [14]:
import torch.nn.functional as F

# Class weights (isteğe bağlı, örnek)
class_weights = torch.tensor([1.0] * 9).cuda()  # şimdilik eşit ağırlık
# İstersen burada data'dan gerçek orana göre ayarlama yaparız.

criterion = nn.CrossEntropyLoss(
    weight=class_weights,
    ignore_index=255  # valid olmayan pikseller hesaba katılmaz
)


In [15]:
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.01,
    momentum=0.9,
    weight_decay=5e-4
)

# Poly LR scheduler
def poly_lr_scheduler(optimizer, init_lr, iter, max_iter, power=0.9):
    new_lr = init_lr * (1 - iter / max_iter) ** power
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_lr


In [16]:
from tqdm import tqdm

def train_one_epoch(model, dataloader, optimizer, criterion, device, epoch, max_epoch):
    model.train()
    running_loss = 0.0
    pbar = tqdm(dataloader)

    for i, batch in enumerate(pbar):
        if batch is None:
            continue  # batch komple None olabilir

        images, masks = batch

        if images is None or masks is None:
            continue

        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()

        # LR update
        iter_num = epoch * len(dataloader) + i
        max_iter = max_epoch * len(dataloader)
        poly_lr_scheduler(optimizer, init_lr=0.01, iter=iter_num, max_iter=max_iter)

        pbar.set_description(f"Epoch {epoch} Loss: {running_loss/(i+1):.4f}")

    return running_loss / len(dataloader)


In [17]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)


# Val dataset ve loader (istersen burada da tanımlayabiliriz)
val_dataset = AgricultureDataset(
    image_dir_rgb="/content/veri_seti/Agriculture-Vision-2021/val/images/rgb",
    image_dir_nir="/content/veri_seti/Agriculture-Vision-2021/val/images/nir",
    label_dir="/content/veri_seti/Agriculture-Vision-2021/val/combined_labels",
    mask_dir="/content/veri_seti/Agriculture-Vision-2021/val/masks",
    transform=transform  # val için de aynı transform kullanıyoruz (augmentations OFF olmalı idealde)
)

val_loader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


📊 Dataset'te kullanılabilir örnek sayısı: 18334


In [72]:
!ls /content/veri_seti/Agriculture-Vision-2021/train/combined_labels | head

11IE4DKTR_11556-9586-12068-10098.png
11IE4DKTR_6121-684-6633-1196.png
11TAQJ6ET_1124-8510-1636-9022.png
11TAQJ6ET_1148-8089-1660-8601.png
11TAQJ6ET_1288-961-1800-1473.png
11TAQJ6ET_1422-8427-1934-8939.png
11TAQJ6ET_1567-8177-2079-8689.png
11TAQJ6ET_1749-948-2261-1460.png
11TAQJ6ET_1995-8426-2507-8938.png
11TAQJ6ET_2034-926-2546-1438.png


In [81]:
!ls /content/veri_seti/Agriculture-Vision-2021/train/images/rgb | head


11IE4DKTR_11556-9586-12068-10098.jpg
11IE4DKTR_6121-684-6633-1196.jpg
11TAQJ6ET_1124-8510-1636-9022.jpg
11TAQJ6ET_1148-8089-1660-8601.jpg
11TAQJ6ET_1288-961-1800-1473.jpg
11TAQJ6ET_1422-8427-1934-8939.jpg
11TAQJ6ET_1567-8177-2079-8689.jpg
11TAQJ6ET_1749-948-2261-1460.jpg
11TAQJ6ET_1995-8426-2507-8938.jpg
11TAQJ6ET_2034-926-2546-1438.jpg


In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 20  # Paper'da yaklaşık 25k iterasyon yapılıyordu.



In [26]:
import torch.nn as nn

# Her sınıf için örnek weight önerisi (orantılı ağırlıklar)
class_weights = torch.tensor([
    1.0,  # class 0 (genellikle background, yok)
    2.0,  # double_plant
    6.0,  # drydown
    6.0,  # endrow
    4.0,  # nutrient_deficiency
    10.0, # planter_skip (çok nadir)
    3.0,  # storm_damage
    5.0,  # water
    6.0   # waterway
], dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights, ignore_index=255)


In [27]:
for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device, epoch, num_epochs)
    print(f"📘 Epoch {epoch} Loss: {train_loss:.4f}")


Epoch 0 Loss: 0.9898: 100%|██████████| 14236/14236 [37:23<00:00,  6.34it/s]


📘 Epoch 0 Loss: 0.9898


Epoch 1 Loss: 0.9297: 100%|██████████| 14236/14236 [37:25<00:00,  6.34it/s]


📘 Epoch 1 Loss: 0.9297


Epoch 2 Loss: 0.9096: 100%|██████████| 14236/14236 [37:23<00:00,  6.34it/s]


...

📘 Epoch 17 Loss: 0.6081


Epoch 18 Loss: 0.5584: 100%|██████████| 14236/14236 [37:23<00:00,  6.35it/s]


📘 Epoch 18 Loss: 0.5584


Epoch 19 Loss: 0.4961: 100%|██████████| 14236/14236 [37:23<00:00,  6.35it/s]

📘 Epoch 19 Loss: 0.4961


In [28]:
def compute_iou(preds, labels, num_classes=9, ignore_index=255):
    ious = []
    preds = preds.view(-1)
    labels = labels.view(-1)

    for cls in range(num_classes):
        pred_inds = preds == cls
        target_inds = labels == cls

        if target_inds.sum() == 0:
            ious.append(float('nan'))  # bu sınıf yoksa NaN ekle
            continue

        intersection = (pred_inds[target_inds]).long().sum().item()
        union = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection

        if union == 0:
            ious.append(float('nan'))
        else:
            ious.append(intersection / union)

    return ious


In [29]:
from tqdm import tqdm

def validate(model, val_loader, device, num_classes=9):
    model.eval()
    iou_list = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            ious = compute_iou(preds, labels, num_classes)
            iou_list.append(ious)

    iou_list = torch.tensor(iou_list)
    mean_ious = torch.nanmean(iou_list, dim=0)  # Her sınıfın ortalama IoU'su
    miou = torch.nanmean(mean_ious)  # Ortalama mIoU

    return mean_ious, miou


In [30]:
mean_ious, miou = validate(model, val_loader, device)

for idx, class_iou in enumerate(mean_ious):
    print(f"Sınıf {idx} IoU: {class_iou:.4f}")

print(f"📊 Ortalama mIoU: {miou:.4f}")


100%|██████████| 2292/2292 [04:09<00:00,  9.19it/s]

Sınıf 0 IoU: 0.6447
Sınıf 1 IoU: 0.5157
Sınıf 2 IoU: 0.2320
Sınıf 3 IoU: 0.3404
Sınıf 4 IoU: 0.2189
Sınıf 5 IoU: 0.0478
Sınıf 6 IoU: 0.3902
Sınıf 7 IoU: 0.1895
Sınıf 8 IoU: 0.2383
📊 Ortalama mIoU: 0.3131


In [31]:
# Modeli kaydet (.pth)
torch.save(model.state_dict(), "fpn_model_rgbnir_epoch20.pth")
print("✅ Model başarıyla kaydedildi: fpn_model_rgbnir_epoch20.pth")


✅ Model başarıyla kaydedildi: fpn_model_rgbnir_epoch20.pth


In [32]:
model = FPNModel(num_classes=9, in_channels=4).to(device)
model.load_state_dict(torch.load("fpn_model_rgbnir_epoch20.pth"))
model.eval()


FPNModel(
  (layer1): Sequential(
    (0): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 

In [33]:
import os
os.makedirs("test_predictions", exist_ok=True)


In [34]:
from tqdm import tqdm
import cv2
import numpy as np
import torch

def predict_and_save_masks(model, test_loader, device, save_dir="test_predictions"):
    model.eval()
    with torch.no_grad():
        for i, (images, _) in enumerate(tqdm(test_loader)):
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()

            for b in range(images.shape[0]):
                pred_mask = preds[b].astype(np.uint8)  # [H, W]
                save_path = os.path.join(save_dir, f"pred_{i*test_loader.batch_size + b}.png")
                cv2.imwrite(save_path, pred_mask)

    print(f"✅ Tahmin maskeleri {save_dir} klasörüne kaydedildi.")


In [35]:
from torch.utils.data import Dataset
import os
import cv2
import numpy as np
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pathlib import Path

class TestAgricultureDataset(Dataset):
    def __init__(self, image_dir_rgb, image_dir_nir, transform=None):
        self.image_dir_rgb = image_dir_rgb
        self.image_dir_nir = image_dir_nir
        self.transform = transform

        # Ortak dosya köklerini bul
        rgb_files = {Path(f).stem: f for f in os.listdir(image_dir_rgb)}
        nir_files = {Path(f).stem: f for f in os.listdir(image_dir_nir)}
        self.common_stems = sorted(list(set(rgb_files.keys()) & set(nir_files.keys())))

    def __len__(self):
        return len(self.common_stems)

    def __getitem__(self, idx):
        stem = self.common_stems[idx]
        rgb_path = os.path.join(self.image_dir_rgb, f"{stem}.jpg")
        nir_path = os.path.join(self.image_dir_nir, f"{stem}.jpg")

        rgb = cv2.imread(rgb_path)[:, :, ::-1]
        nir = cv2.imread(nir_path, cv2.IMREAD_GRAYSCALE)
        nir = np.expand_dims(nir, axis=-1)
        image = np.concatenate([rgb, nir], axis=-1)

        if self.transform:
            image = self.transform(image=image)["image"]

        return image, stem  # stem = dosya adı


In [36]:
# Test için transform (augmentation YOK!)
test_transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize(),
    ToTensorV2()
])

test_dataset = TestAgricultureDataset(
    image_dir_rgb="/content/veri_seti/Agriculture-Vision-2021/test/images/rgb",
    image_dir_nir="/content/veri_seti/Agriculture-Vision-2021/test/images/nir",
    transform=test_transform
)

from torch.utils.data import DataLoader
test_loader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [37]:
def predict_and_save_masks(model, test_loader, device, save_dir="test_predictions"):
    os.makedirs(save_dir, exist_ok=True)
    model.eval()
    with torch.no_grad():
        for images, stems in tqdm(test_loader):
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()

            for b in range(images.shape[0]):
                pred_mask = preds[b].astype(np.uint8)
                save_path = os.path.join(save_dir, f"{stems[b]}.png")
                cv2.imwrite(save_path, pred_mask)

    print(f"✅ Tahminler {save_dir} klasörüne kaydedildi.")


In [38]:
predict_and_save_masks(model, test_loader, device)


100%|██████████| 4927/4927 [04:15<00:00, 19.32it/s]

✅ Tahminler test_predictions klasörüne kaydedildi.


In [39]:
!zip -r test_predictions.zip test_predictions


Streaming output truncated to the last 5000 lines.
  adding: test_predictions/8HWHY2JBM_2168-12305-2680-12817.png (deflated 92%)
  adding: test_predictions/ZQABXWERC_6724-3131-7236-3643.png (deflated 9%)
  adding: test_predictions/W73BID9QF_4731-8747-5243-9259.png (deflated 90%)
  adding: test_predictions/MYJE6E87W_5568-2299-6080-2811.png (deflated 13%)
  adding: test_predictions/P9THUZUQ3_11776-16470-12288-16982.png (deflated 25%)
  adding: test_predictions/QA1G7CUCC_1539-3866-2051-4378.png (deflated 46%)
  adding: test_predictions/D818WD7LL_1841-3332-2353-3844.png (deflated 25%)
  adding: test_predictions/YU2FYXTPV_498-3315-1010-3827.png (deflated 25%)
  adding: test_predictions/PU9ELGCFZ_8115-1689-8627-2201.png (deflated 16%)
  adding: test_predictions/2CQNIMUUU_1229-7921-1741-8433.png (deflated 24%)
  adding: test_predictions/AGREYD9H3_7133-6207-7645-6719.png (deflated 38%)
  adding: test_predictions/7PFCT77EZ_5884-1183-6396-1695.png (deflated 63%)
  adding: test_predictions/KUVMYH

In [49]:
import pandas as pd

def rle_encode(mask):
    pixels = mask.flatten(order='F')  # sütun bazlı flatten
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    if len(runs) % 2 != 0:
        runs = np.append(runs, len(pixels)-2)
    runs[1::2] -= runs[::2]
    return " ".join(str(x) for x in runs)


submission_data = []

for stem in sorted(os.listdir("test_predictions")):
    filepath = os.path.join("test_predictions", stem)
    mask = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

    # Sadece bir sınıf (örneğin "water" sınıfı) için encode ediyorsak filtrele:
    # binary_mask = (mask == 6).astype(np.uint8)  # Sınıf 6 örneğin "water"
    # encoded = rle_encode(binary_mask)

    # Çok sınıflı segmentasyon için her pikseli encode et:
    encoded = rle_encode(mask)

    submission_data.append({
        "filename": stem,
        "encoded_pixels": encoded
    })

df = pd.DataFrame(submission_data)
df.to_csv("submission.csv", index=False)
print("✅ submission.csv oluşturuldu.")


✅ submission.csv oluşturuldu.


In [50]:
df.to_csv("/content/drive/MyDrive/submission.csv", index=False)


In [51]:
print(df.head())


                            filename  \
0  17FCUVCMI_1728-7254-2240-7766.png   
1  17FCUVCMI_1995-5512-2507-6024.png   
2  17FCUVCMI_3873-3301-4385-3813.png   
3  17FCUVCMI_3875-3011-4387-3523.png   
4  17FCUVCMI_3895-2473-4407-2985.png   

                                      encoded_pixels  
0  119593 5 120074 3 120090 2 120104 8 120569 4 1...  
1  109319 24 109828 28 110337 32 110845 38 111354...  
2  123570 1 124073 13 124087 5 124584 22 125094 2...  
3  124674 10 124698 3 125177 21 125206 10 125686 ...  
4                                                     
